In [ ]:
%%capture
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

In [ ]:
df = df.drop(columns = 'Id')
test =test.drop(columns = 'Id')

In [ ]:
X = df.drop(columns = 'Pawpularity')
y = df.Pawpularity

In [ ]:
#create_folds.py 
from sklearn import model_selection
df["kfold"] = -1

df = df.sample(frac=1).reset_index(drop=True)

kf = model_selection.StratifiedKFold(n_splits=5, shuffle=False, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(X=df,y=df.Pawpularity.values)):
    print(len(train_idx), len(val_idx))
    df.loc[val_idx, 'kfold'] = fold


In [ ]:
params = {'learning_rate': 0.2113303692287583,
  'reg_lambda': 0.0005172374569093787,
  'reg_alpha': 0.001273145009879541,
  'subsample': 0.12703520389320402,
  'colsample_bytree': 0.2566392406542385,
  'max_depth': 2}

In [ ]:
import joblib
import lightgbm as lgb
import xgboost as xg
import pandas as pd
from sklearn import metrics


from sklearn import tree
def run(fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    df_train = df_train.drop(columns = 'kfold')
    df_valid = df_valid.drop(columns = 'kfold')
    x_train = df_train.drop('Pawpularity', axis=1).values
    y_train = df_train.Pawpularity.values
    x_valid = df_valid.drop('Pawpularity', axis=1).values
    y_valid = df_valid.Pawpularity.values
    model = xg.XGBRegressor(
        random_state=42,
        tree_method="gpu_hist",
        gpu_id=0,
        n_estimators=10000,
        predictor="gpu_predictor",
        **params,
    )
    model.fit(x_train, y_train)
    y_pred = model.predict(x_valid)
    accuracy_score = np.sqrt(metrics.mean_squared_error(y_valid,y_pred))
    print(f"Fold={fold}, RMSE_score={accuracy_score}")
    File_name = 'model_' + str(fold) + '.pkl'
    joblib.dump(
    model,File_name)
for i in range(5):
    run(fold = i)

In [ ]:
model_0 = joblib.load('./model_0.pkl')
model_1 = joblib.load('./model_1.pkl')
model_2 = joblib.load('./model_2.pkl')
model_3 = joblib.load('./model_3.pkl')
model_4 = joblib.load('./model_4.pkl')

In [ ]:
y_final_3 = model_3.predict(test)
y_final_0 = model_0.predict(test)
y_final_1 = model_1.predict(test)
y_final_2 = model_2.predict(test)
y_final_4 = model_4.predict(test)

In [ ]:
y_final_avg = (y_final_0 + y_final_1 +y_final_2 + y_final_3 + y_final_4)/5

#####  IMAGE ONE